# Causal Graph Visualization

- Bokeh
- pivis

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import networkx as nx

from bokeh.io import push_notebook, output_notebook, show, reset_output
from bokeh.plotting import figure, from_networkx

G = nx.karate_club_graph()
output_notebook()

Loading BokehJS ...

In [3]:
from multiprocessing import cpu_count
import sys

sys.path.append('../')
from tsdr import tsdr
from diagnoser import diag
from meltria import loader
import diagnoser.metric_node as mn

filepath = '/datasets/argowf-chaos-hg68n/2021-12-09-argowf-chaos-hg68n-user_pod-memory-hog_0.json'
data_df, mappings = loader.read_metrics_file(filepath)

reducer = tsdr.Tsdr(tsdr.ar_based_ad_model, **{
    'tsifter_step1_ar_regression': 'n',
    'tsifter_step1_ar_anomaly_score_threshold': 0.01,
    'tsifter_step1_cv_threshold': 0.05,
    'tsifter_step1_ar_dynamic_prediction': False,
    'tsifter_step2_clustering_threshold': 0.01,
    'tsifter_step2_clustered_series_type': 'raw',
    'tsifter_step2_clustering_dist_type': 'sbd',
    'tsifter_step2_clustering_choice_method': 'medoid',
    'tsifter_step2_clustering_linkage_method': 'single',
})
_, reduced_df_by_step, metrics_dimension, _ = reducer.run(
    series=data_df,
    max_workers=cpu_count(),
)
reduced_df = reduced_df_by_step['step2']
causal_graph, stats = diag.run(
    reduced_df, mappings, **{
        'pc_library': 'pcalg',
        'pc_citest': 'fisher-z',
        'pc_citest_alpha': 0.01,
        'pc_variant': None,
    }
)
print(len(causal_graph.nodes))
relabeled_mapping = mn.MetricNodes.from_list_of_metric_node(list(causal_graph.nodes)).node_to_label()
print(relabeled_mapping)
relabeled_graph = nx.relabel_nodes(causal_graph, relabeled_mapping, copy=True)

16
{<diagnoser.metric_node.MetricNode object at 0x7ff6d15b1040>: 'c-front-end_network_receive_bytes_total', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b1250>: 'c-front-end_memory_usage_bytes', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b1820>: 'c-carts_cpu_cfs_throttled_periods_total', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b10d0>: 'c-front-end_sockets', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b4c10>: 'c-orders_cpu_cfs_throttled_seconds_total', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b4f10>: 'c-orders_cpu_cfs_throttled_periods_total', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b4dc0>: 'c-orders_cpu_cfs_periods_total', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b4190>: 'c-user_sockets', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b48b0>: 'c-user_threads', <diagnoser.metric_node.MetricNode object at 0x7ff6d15b48e0>: 'c-user_last_seen', <diagnoser.metric_node.MetricNode object at 0x7ff6d15bf2e0>: 's-f

In [4]:
# from bokeh.models import (BoxZoomTool, Circle, HoverTool,
#                           MultiLine, Plot, Range1d, ResetTool, NodesAndLinkedEdges)
# from bokeh.palettes import Spectral4


# HOVER_TOOLTIPS = [("Metric", "@index")]
# p = figure(title="caulsal graph visualization", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
#             tooltips = HOVER_TOOLTIPS, tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom')
# gr = from_networkx(relabeled_graph, nx.spring_layout, scale=1, center=(0,0))
# gr.node_renderer.glyph = Circle(size=20, fill_color=Spectral4[0])
# gr.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
# gr.inspection_policy = NodesAndLinkedEdges()
# p.renderers.append(gr)

In [5]:
from pyvis.network import Network
from IPython.display import IFrame, HTML, display_html

for node in causal_graph.nodes:
    if node.is_service():
        color = "blue"
        size = 20
    elif node.is_container():
        color = "green"
        size = 10
    elif node.is_middleware():
        color = "purple"
        size = 10
    else:
        color = "grey"
        size = 10
    if node.is_root():
        color = "red"
        size = 25
    causal_graph.nodes[node]["color"] = color
    causal_graph.nodes[node]["size"] = size
    causal_graph.nodes[node]["label"] = node.label

## Set arrows

nwg = Network(notebook=True, directed=True)
relabeled_mapping = mn.MetricNodes.from_list_of_metric_node(list(causal_graph.nodes)).node_to_label()
relabeled_graph = nx.relabel_nodes(causal_graph, relabeled_mapping, copy=True)
nwg.from_nx(relabeled_graph)
nwg.toggle_physics(True)

In [7]:
from bs4 import BeautifulSoup
import numpy as np
from IPython.display import IFrame, HTML, display_html

from bokeh import plotting
from bokeh.embed import components
from bokeh.resources import CDN

with open('../tmp/nw.html') as f:
    soup = BeautifulSoup(f.read(), "html.parser")
    p: plotting.Figure = plotting.figure(
        title=f"sample",
        x_axis_label='interval', y_axis_label='zscore',
    )
    for node in causal_graph.nodes:
        series: np.ndarray = data_df[node.label].to_numpy()
        if node.is_root():
            p.line(np.arange(series.size), series, legend_label=f"root metric: {node.label}", line_width=1)
        else:
            p.line(np.arange(series.size), series, legend_label=f"{node.label}", line_width=1)
    script, div = components(p)
    body = soup.find('body')
    pyvis_div = soup.find('div', id="mynetwork")
    pyvis_div.insert_after(BeautifulSoup(div, "html.parser"))
    body.append(BeautifulSoup(script, "html.parser"))
t = show(p, notebook_handle=True)

RuntimeError: Models must be owned by only a single document, LegendItem(id='1072', ...) is already in a doc